If labels are engines and the binary columns are tests on those engines... Then we are just creating a 'autocorrelation'  effect ?
---
suppose the codes are the 'engines types Diesel-Gas', 'the shiftgearbox types Automatic 9G 7G', 'and the binary columns are emission test CO, NO, CH
Then automatically all the tests using the label data are autocorrelating ?

__________
**

corr = 0.6
----------

**
_________

In [ ]:
#Lasso
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder
from sklearn import linear_model
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.cm as cm


#open data
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
# y values drop 'y' splitsen,
y_train = train['y']
x_train=train.drop(['y'],axis=1)

# concat everything, alles samenhangen
x_tota=x_train.append(test)

# drop ID since this can autocorrelate too... (although mercedes is making millions of cars a year, i don't think that 8000 is representative for their annualfleet) splitsen
ID_tota=x_tota['ID']
x_tota=x_tota.drop(['ID'],axis=1)
x_tota = x_tota.reset_index(drop=True)
print(x_tota.shape)

#transform labels to values... waarden maken
for c in x_tota.columns:
    if x_tota[c].dtype == 'object':
        lbl = LabelEncoder() 
        lbl.fit(list(x_tota[c].values) ) 
        x_tota[c] = lbl.transform(list(x_tota[c].values))
labellist=['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8']

# seperate labellist
y_=x_tota[labellist]
x_=x_tota.drop(labellist,axis=1)
print(x_.shape)

#do a regression
reg = linear_model.Ridge (alpha = .5)
#reg = linear_model.Lasso(alpha = 0.1)
reg.fit(x_,y_)  
print('R2',reg.score(x_, y_))  #lasso 0.5
results=pd.DataFrame(reg.coef_)
resulT=results.T
resulT.columns=labellist
print(results)

do we uncorrelate the data with a PCA ? what do you think ?

Some nearly constant columns
---

In [ ]:
traind=train.describe()
traind=traind.drop('y',axis=1)
testd=test.describe()

totald=traind.append(testd)
totaldT=totald.T
totaldT=totaldT.drop(['count','std','min','25%','50%','75%','max'],axis=1)
totaldT.columns=['m1','m2']

def ratio(m1,m2):
    if m2>0:
        ratio=m1/m2
    else:
        ratio=0
    return ratio


totaldT['mratio']=totaldT[['m1','m2']].apply(lambda x: ratio(*x), axis=1)
print(totaldT[totaldT['m1']>0.96])
print(totaldT[totaldT['m2']>0.96])

print(totaldT.sort_values(by='mratio'))

In [ ]:
usable_columns = list(set(train.columns) - set(['ID', 'y','X0', 'X6', 'X3', 'X4', 'X8', 'X2', 'keys', 'X5', 'X1']))

vcovm=np.cov(train[usable_columns].T,train['y'].T)
print(vcovm)
df_vcm=pd.DataFrame(vcovm)
diagonalen=pd.DataFrame(np.diag(df_vcm))
diagonalen.columns=['dia']
print(diagonalen[diagonalen['dia']>5])
print(train[usable_columns].T.dot(train[usable_columns]))

#covmx=np.cov(np.linalg.inv(train[usable_columns].T.dot(train[usable_columns])))
#print(covmx)
#diagxx=pd.DataFrame(np.diag(covmx))
#print(diagxx)